Creating Tokens

In [ ]:
#download the verdict doc into this code window
with open("the-verdict.txt", 'r', encoding="utf-8") as f:
    raw_text = f.read() #the raw_text variable s the variable we are storing the data in the file in

#printing the total number of characters in the file
print(f'Total number of Character: {len(raw_text)}')
#print the fiart 100 characters in the file
print(raw_text[:99])

Total number of Character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [ ]:
import re
#tokenicing the raw text
#this splots the text in the raw text based on , . : ; ? _ ! ' --  () " and space
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
#this removes every trailing white space from the tokens
preprocessed = [item.strip() for item in preprocessed if item.strip()]
#prin the first 31 items
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [6]:
#print the lenght of the tokenized word
print(len(preprocessed))

4690


Converting tokens into token IDs

In [ ]:
#create a collectiong of the unique tokens and sort it alphabetically, 
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


A vocabulary is a dictionary that contains each token and its associated token ID

In [8]:
#creting the vocabulary
vocab = {token:integer for integer, token in enumerate(all_words)}

In [9]:
#check out the first 50 items in the vocabulary
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [10]:
#creating a tokenizer class with tow methods, encode and decode
class SimpleTokenizerV1:
    #the vocabulary is the input of the  init method
    def __init__(self, vocab):
        self.str_to_int = vocab
        #extracts the id for each token adn records it as the key and the token as the value
        self.int_to_str = {i:s for s,i in vocab.items()} 

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        #takes the tokens in the preprocessed list and converts them to token ids based on the vocabulary
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        #replace spaces before the specified punctuations in the final text
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [15]:
#instaitate the tokenizer object from the class
tokenizer = SimpleTokenizerV1(vocab)

#text to encode
text = """"It's the last he painted, you know,"
    Mrs. Gisburn said with pardonable pride."""

ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [16]:
#decode the token ids back to tokens
tokens = tokenizer.decode(ids)
print(tokens)

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


Running a text that isn't present in the vocabulary returns an error

In [ ]:
txt = 'Hello, can I have some tea?'
print(tokenizer.encode(txt))

KeyError: 'Hello'

In [18]:
#modifying the vocabulary to include two more tokens for unknown text and end of text
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}

In [19]:
print(len(vocab))

1132


In [21]:
 #print the last five entries of the vocabulary
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [22]:
#extending the simple tokenizer class
class SimpleTokenizerV2:
    #the vocabulary is the input of the  init method
    def __init__(self, vocab):
        self.str_to_int = vocab
        #extracts the id for each token adn records it as the key and the token as the value
        self.int_to_str = {i:s for s,i in vocab.items()} 

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        #if the item is not found in the vocabulary them asiign it the unknown token
        preprocessed = [
            item if item in self.str_to_int else "<|unk|>" for item in preprocessed
        ]
        #takes the tokens in the preprocessed list and converts them to token ids based on the vocabulary
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        #replace spaces before the specified punctuations in the final text
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [25]:
#instantitate the tokeneizer

tokenizer2 = SimpleTokenizerV2(vocab)
txt1 = "Hello, do you like tea?"
txt2 = "In the sunlit terraces of the palace."
txt3 = " <|endoftext|> ".join((txt1, txt2))

print(txt3)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [27]:
ids = tokenizer2.encode(txt3)
print(ids)
print(tokenizer2.decode(ids))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


In [29]:
import tiktoken
#instatiate the byte pair encoding tokenizer in tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [30]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
)
integers = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [31]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.
